# Limpieza inicial
En este notebook se realiza una limpieza inicial de los registros que presentan inconsistencias en algún campo. Las inconsistencias se detallan en el análisis exploratorio.

 En general los criterios de eliminación son: 
- Categorías en variables socio-económicas inexistentes.
- Montos de deuda negativos.
- Montos de pagos y deudas mayores al cupo de la tarjeta.

In [1]:
import pandas as pd

In [2]:
# Cagar los datos provenientes de los archivos csv
transacciones = pd.read_csv("data/historial_transacciones.csv", delimiter="|")
clientes = pd.read_csv("data/informacion_clientes.csv", delimiter="|")
etiquetas = pd.read_csv("data/etiquetas.csv", delimiter="|")

## Eliminar datos socio económicos inconsistentes

| **Campo**                            | **Descripción**                                                                                                                                | **Comentario**                                                                           |
|--------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| SEX (nominal)                        | Sexo biológico del cliente.  <br>Categorías: Masculino , Femenino                                                                                  | No presenta datos inconsistentes                                                         |
| EDUCATION (nominal)                  | Nivel de escolaridad del cliente <br>Categorías: colegio, universidad, posgrado, otro                                                              | Valores inconsistentes "0","5","6"                                                       |
| MARRIAGE (nominal)                   | Estado civil del cliente <br>Categorías: soltero_a, casado, otro                                                                                   | Valores inconsistentes  "0"                                                              |


In [3]:
def limpiar_clientes(df_clientes):

    df_limpio = df_clientes.copy()

    # Convertir en minúsculas todos los valores de las columnas categóricas
    categoricas = ["EDUCATION", "MARRIAGE", "SEX"]
    for var in categoricas:
        df_limpio[var] = df_limpio[var].map(lambda x: x.lower())

    # Eliminar los registros con valores inconsistentes en las columnas categóricas
    cat_edu = ["universidad", "colegio", "posgrado", "otro"]
    cat_marr = ["casado", "soltero_a", "otro"]
    df_limpio = df_limpio.query('EDUCATION in @cat_edu and MARRIAGE in @cat_marr').copy()

    # Convertir en series de tipo categóricas a las columnas categóricas
    for var in categoricas:
        df_limpio[var] = df_limpio[var].astype('category')
    
    # Calcular e imprimir el número de registros eliminados
    num_reg_eliminados = len(df_clientes) - len(df_limpio)
    print(f"Se eliminaron {num_reg_eliminados} registros con valores inconsistentes")

    return df_limpio.reset_index(drop=True)

In [4]:
clientes_cleaned = limpiar_clientes(clientes)

Se eliminaron 331 registros con valores inconsistentes


## Eliminar inconsistencias monetarias

Se eliminan registros que contengas montos negativos de deudas en algún mes y montos de pagos o deudas que superen al cupo inicial de la tarjeta

In [5]:
def transformar_transacciones(transacciones):
    df = transacciones.copy()

    # Cambiar el número del mes por etiquetas temporales de la forma t-i
    month_map = {9-i: f't-{i}' for i in range(6)}
    df["MONTH"] = df["MONTH"].map(month_map)

    # Transponer las observaciones temporales
    df = pd.pivot(df, index="CLIENT_ID", columns="MONTH")

    # Eliminar el multíndice generado por la transposición
    df.columns = df.columns.to_flat_index()
    df.columns = [ f"{var}_{time_tag}" for (var, time_tag) in df.columns ]
    df = df.reset_index()

    return df

In [6]:
# Transponer los meses como columnas para cada variable con marca temporal
transacciones_transf = transformar_transacciones(transacciones)

In [7]:
# Unir la tabla de transacciones y clientes. Se necesita el cupo para identificar los valores monetarios inconsistentes
df_modelo = clientes_cleaned.merge(transacciones_transf, on="CLIENT_ID", how="inner") # Solo se traen los registros limpios de la tabla clientes porque es inner-join

In [8]:
# Hallar índice de la primera columna que tiene un valor monetario
list(df_modelo.columns).index('DEUDA_MES_t-0')

12

In [9]:
# Condición para identificar registros con por lo menos un pago o deuda mayor al cupo
superar_cupo = df_modelo.iloc[:, 12:].gt(df_modelo['Cupo_TC'], axis=0).any(axis=1)
# Eliminar registros con inconsistencias monetarias
df_modelo = df_modelo[~superar_cupo]

# Identificar registro con por lo menos un monto negativo de pago o deuda 
trans_negativa =  (df_modelo.iloc[:, 12:]<0).any(axis=1)
df_modelo = df_modelo[~trans_negativa]

In [10]:
reg_eliminados = len(clientes) - len(df_modelo)
prop_inconsistentes = reg_eliminados / len(clientes)
print("Se eliminaron en total {} registros con datos inconsistentes.\nLo que respresenta un {:.2%} de la muestra".format(reg_eliminados, prop_inconsistentes))

Se eliminaron en total 4905 registros con datos inconsistentes.
Lo que respresenta un 20.44% de la muestra


## Unir tablas

In [11]:
# Traer etiquetas
df_modelo = df_modelo.merge(etiquetas, on="CLIENT_ID", how="inner")
# Eliminar la columna Id ya que ahora no es necesaria
df_modelo = df_modelo.drop(columns="CLIENT_ID")

In [12]:
display(df_modelo)

,Cupo_TC,SEX,EDUCATION,MARRIAGE,AGE,RETRASO_PAGO_ESTADO_t-0,RETRASO_PAGO_ESTADO_t-1,RETRASO_PAGO_ESTADO_t-2,RETRASO_PAGO_ESTADO_t-3,RETRASO_PAGO_ESTADO_t-4,...,DEUDA_MES_t-3,DEUDA_MES_t-4,DEUDA_MES_t-5,PAGO_MES_t-0,PAGO_MES_t-1,PAGO_MES_t-2,PAGO_MES_t-3,PAGO_MES_t-4,PAGO_MES_t-5,default payment next month
0,80000,femenino,colegio,soltero_a,52,0,0,0,0,0,...,30470,18306,7432,1623,1740,1200,366,7432,0,0
1,210000,femenino,universidad,casado,36,0,0,0,0,0,...,148190,147993,131629,6483,5082,6000,5900,4544,4700,0
2,230000,femenino,colegio,casado,44,-1,-1,-1,-1,-1,...,6222,15121,17425,11632,4987,6222,15121,17425,17007,0
3,130000,femenino,posgrado,casado,36,0,0,0,0,0,...,94623,90556,92394,5000,4400,5000,4000,3500,4000,0
4,470000,femenino,universidad,casado,35,0,0,0,0,0,...,61500,59563,57368,2505,3000,3016,2200,3000,1600,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19090,280000,masculino,universidad,soltero_a,45,0,0,-1,0,0,...,136249,111454,96425,7000,208141,5003,5002,5000,3000,0
19091,180000,femenino,universidad,casado,46,1,2,0,0,0,...,26574,23123,30680,1711,1300,7504,1123,13000,0,0
19092,180000,masculino,universidad,casado,35,1,2,0,0,0,...,33370,33370,0,0,1400,2200,0,0,0,0
19093,320000,masculino,universidad,soltero_a,30,0,0,0,0,0,...,26875,26318,25880,1353,1224,739,742,751,703,0


## Desbalance de clase

In [13]:
df_modelo["default payment next month"].value_counts()

default payment next month
0    15022
1     4073
Name: count, dtype: int64

In [14]:
prop_desbalance = df_modelo["default payment next month"].value_counts()[1] / len(df_modelo)
print("El porcentaje de impagos el próximo mes es de {:.2%}".format(prop_desbalance))

El porcentaje de impagos el próximo mes es de 21.33%


El desbalance de clase de la variable objetivo varía un $0.67\%$

## Exportara conjunto de datos limpio

In [15]:
df_modelo.to_csv("cleaned_data/limpio_inicial.csv", index=False)